<a href="https://colab.research.google.com/github/faizaungguh/e-ticket-generator/blob/main/e_Ticket_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# e-Ticket Generator

## Persiapan Folder Eksekusi

### Izinkan Akses Folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive berhasil terhubung!")

Mounted at /content/drive

✅ Google Drive berhasil terhubung!


### Install Library

In [ ]:
# Sel 2: Instal Library & Instal Font (via apt-get)
!pip install pandas qrcode[pil] -q

print("Menginstal font Roboto (via apt-get)...")
# Menggunakan manajer paket Linux untuk menginstal font secara sistem
!apt-get install fonts-roboto -y -q

print("✅ Library pandas, qrcode, dan font Roboto (via system) telah siap.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
Menginstal font Roboto (via apt-get)...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  fonts-roboto-unhinted
The following NEW packages will be installed:
  fonts-roboto fonts-roboto-unhinted
0 upgraded, 2 newly installed, 0 to remove and 41 not upgraded.
Need to get 2,380 kB of archives.
After this operation, 6,405 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-roboto-unhinted all 2:0~20170802-3 [2,376 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-roboto all 2:0~20170802-3 [3,212 B]
Fetched 2,380 kB in 0s (14.4 MB/s)
Selecting previously unselected package fonts-roboto-unhinted.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../fonts-roboto-unhinted_2%3a0~20170802-3_all.deb ...
Unpacking fo

## Atur Path dan file

### Masuk ke Lokasi csv Disimpan

In [ ]:
import os

# Ganti path ini agar sesuai dengan lokasi folder Anda di Google Drive
WORKING_DIR = "/content/drive/MyDrive/G Colab/e-Ticket Pramuka Garuda"

try:
    os.chdir(WORKING_DIR)
    print(f"✅ Berhasil pindah direktori ke: {os.getcwd()}")
    print("Sekarang skrip akan membaca dan menulis file dari folder ini.")
except FileNotFoundError:
    print(f"❌ ERROR: Folder tidak ditemukan di: {WORKING_DIR}")
    print("Harap periksa kembali ejaan nama folder dan path Anda.")
except Exception as e:
    print(f"❌ ERROR: {e}")

✅ Berhasil pindah direktori ke: /content/drive/MyDrive/G Colab/e-Ticket Pramuka Garuda
Sekarang skrip akan membaca dan menulis file dari folder ini.


In [ ]:
ls

 final-entry.csv  'template-ticket (1).png'   template-ticket.png


### Penyederhanaan data registrasi

#### Atur File mana

In [ ]:
# ==========================================
# --- 1. KONFIGURASI (WAJIB DIISI) ---
# ==========================================

# Nama file INPUT (Pastikan nama file ini BENAR dan ada di folder yang sama)
INPUT_FILE = "final-entry.csv"  # <-- Sesuaikan nama file Anda

# Nama file OUTPUT
OUTPUT_FILE = "data-eksekusi.csv"

# Definisi Nama Kolom di File Mentah
KOLOM_TIMESTAMP_ASLI = 'Timestamp'
KOLOM_EMAIL_ASLI     = 'Username'      # Cek header CSV asli Anda, apakah 'Username' atau 'Email Address'?
KOLOM_NAMA_ASLI      = 'Nama Lengkap'
KOLOM_KATEGORI_ASLI  = 'Tribun yang Kakak Pilih'

# Mapping Kategori -> Tribun
TRIBUNE_MAP = {
    "CAT 1": "A",
    "CAT 2": "B",
    "CAT 3": "C1",
    "CAT 4": "C2",
    "CAT 5": "D"
}

# Mapping Kategori -> Gate
GATE_MAP = {
    "CAT 1": "7",
    "CAT 2": "2",
    "CAT 3": "2",
    "CAT 4": "3",
    "CAT 5": "3"
}


#### Eksekusi Penyederhanaan Data

In [ ]:
import pandas as pd
import os
import secrets  # <--- TAMBAHAN: Untuk membuat token acak yang aman
import string   # <--- TAMBAHAN: Untuk mengambil daftar huruf/angka

# ==========================================
# --- FUNGSI TAMBAHAN: GENERATE TOKEN ---
# ==========================================
def generate_unique_token(length=8):
    """
    Membuat string acak (huruf besar/kecil + angka) dengan panjang tertentu.
    Contoh output: 'a7B9xYz1'
    """
    alphabet = string.ascii_letters + string.digits
    return ''.join(secrets.choice(alphabet) for _ in range(length))

# ==========================================
# --- 2. PROSES EKSEKUSI ---
# ==========================================

print(f"📂 Membaca file: {INPUT_FILE}...")

try:
    # Baca CSV
    df = pd.read_csv(INPUT_FILE)

    # --- A. Sortir berdasarkan Timestamp ---
    if KOLOM_TIMESTAMP_ASLI in df.columns:
        df[KOLOM_TIMESTAMP_ASLI] = pd.to_datetime(df[KOLOM_TIMESTAMP_ASLI], errors='coerce')
        df = df.sort_values(by=KOLOM_TIMESTAMP_ASLI)
    else:
        print(f"⚠️ Peringatan: Kolom timestamp '{KOLOM_TIMESTAMP_ASLI}' tidak ditemukan. Data tidak disortir.")

    # --- B. Buat DataFrame baru ---
    df_new = pd.DataFrame()

    # Ambil kolom dengan aman
    df_new['email']    = df.get(KOLOM_EMAIL_ASLI, pd.Series([""]*len(df))).astype(str).str.strip()
    df_new['fullname'] = df.get(KOLOM_NAMA_ASLI, pd.Series([""]*len(df))).astype(str).str.strip()
    df_new['category'] = df.get(KOLOM_KATEGORI_ASLI, pd.Series([""]*len(df))).astype(str).str.strip()

    # --- C. Mapping Gate & Tribun ---
    df_new['tribune'] = df_new['category'].map(TRIBUNE_MAP).fillna("X")
    df_new['gate']    = df_new['category'].map(GATE_MAP).fillna("1")

    # --- D. Status Check-in (Angka 0/1) ---
    df_new['isCheckedIn'] = 0

    # --- E. Buat ID (Format: TRIBUN-0001) ---
    df_new['nomor_urut'] = df_new.groupby('tribune').cumcount() + 1
    df_new['id'] = df_new['tribune'] + '-' + df_new['nomor_urut'].astype(str).str.zfill(4)

    # --- F. GENERATE TOKEN UNIK (Baru) ---
    print("🎲 Sedang men-generate token unik...")
    # Kita pakai list comprehension agar generate unik per baris
    df_new['token'] = [generate_unique_token(8) for _ in range(len(df_new))]

    # --- G. Finalisasi Kolom ---
    # Jangan lupa masukkan 'token' ke dalam list ini
    final_columns = ['id', 'token', 'email', 'fullname', 'tribune', 'category', 'gate', 'isCheckedIn']
    df_output = df_new[final_columns]

    # --- H. Simpan ---
    output_path = os.path.join(os.getcwd(), OUTPUT_FILE)
    df_output.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"\n✅ BERHASIL! File '{OUTPUT_FILE}' telah dibuat.")
    print(f"   Lokasi: {output_path}")
    print("\n--- Preview 5 Baris Pertama ---")
    print(df_output.head().to_markdown(index=False))

except FileNotFoundError:
    print(f"\n❌ ERROR: File '{INPUT_FILE}' tidak ditemukan.")
    print("   -> Cek nama file di bagian KONFIGURASI paling atas script ini.")
except PermissionError:
    print(f"\n❌ ERROR: Izin ditolak. File '{OUTPUT_FILE}' mungkin sedang dibuka di Excel.")
    print("   -> Tutup dulu file Excel-nya lalu coba lagi.")
except Exception as e:
    print(f"\n❌ TERJADI ERROR LAIN: {e}")

📂 Membaca file: final-entry.csv...
🎲 Sedang men-generate token unik...

✅ BERHASIL! File 'data-eksekusi.csv' telah dibuat.
   Lokasi: /content/drive/MyDrive/G Colab/e-Ticket Pramuka Garuda/data-eksekusi.csv

--- Preview 5 Baris Pertama ---
| id      | token    | email   | fullname            | tribune   | category   |   gate |   isCheckedIn |
|:--------|:---------|:--------|:--------------------|:----------|:-----------|-------:|--------------:|
| C1-0001 | N1ssB3t7 |         | Eka destaria        | C1        | CAT 3      |      2 |             0 |
| C1-0002 | mPNlF6jF |         | Fori Yuniana        | C1        | CAT 3      |      2 |             0 |
| A-0001  | fdGSZbQ4 |         | Sukma yudiriantiari | A         | CAT 1      |      7 |             0 |
| D-0001  | THcJmTUD |         | NUR ELIFIA          | D         | CAT 5      |      3 |             0 |
| A-0002  | iHbUUCIg |         | Warsiti             | A         | CAT 1      |      7 |             0 |


## Generate e-Ticket

### Eksekusi

#### Pilih File Eksekusi

In [ ]:
# --- 1. Pengaturan Path (Relatif) ---
CSV_FILE_PATH = "data-eksekusi.csv"
TEMPLATE_PATH = "template-ticket.png"


#### Generate Ticket Digital

In [ ]:
import pandas as pd
import qrcode
import os
import shutil
from PIL import Image, ImageDraw, ImageFont

# ==========================================
# --- KONFIGURASI PATH (Sesuaikan jika perlu) ---
# ==========================================
# Pastikan variabel CSV_FILE_PATH dan TEMPLATE_PATH sudah didefinisikan di sel sebelumnya
# Contoh:
# CSV_FILE_PATH = 'data-eksekusi.csv' (File hasil generate token tadi)
# TEMPLATE_PATH = 'template_tiket.png'

# ==========================================
# --- 1. SETUP FOLDER OUTPUT ---
# ==========================================
i = 1
while True:
    OUTPUT_DIR_NAME = f"generated {i}"
    if not os.path.exists(OUTPUT_DIR_NAME):
        break
    i += 1

LOCAL_TEMP_DIR = f"/content/temp_generated_{i}"
if os.path.exists(LOCAL_TEMP_DIR):
    shutil.rmtree(LOCAL_TEMP_DIR)
os.makedirs(LOCAL_TEMP_DIR)

print(f"File CSV dibaca dari: {os.path.abspath(CSV_FILE_PATH)}")
print(f"Template dibaca dari: {os.path.abspath(TEMPLATE_PATH)}")
print(f"Hasil disimpan di: {os.path.abspath(OUTPUT_DIR_NAME)}")

# ==========================================
# --- 2. MEMUAT FONT ---
# ==========================================
try:
    # Pastikan file .ttf ada di folder yang sama atau sesuaikan path-nya
    font_ticket_id = ImageFont.truetype("Roboto-Bold.ttf", size=110)
    font_combined_info = ImageFont.truetype("Roboto-Bold.ttf", size=42)
    # Opsional: Font kecil jika ingin menampilkan token di bawah
    font_small = ImageFont.truetype("Roboto-Bold.ttf", size=20)
    print("✅ Font berhasil dimuat.")
except IOError:
    print("❌ Error: File font (Roboto-Bold.ttf) tidak ditemukan!")
    raise

# ==========================================
# --- 3. PROSES GENERATE TIKET ---
# ==========================================
total_rows = 0
success_count = 0
failed_count = 0
failed_list = []

try:
    df = pd.read_csv(CSV_FILE_PATH)

    # Cek apakah kolom token ada? Jika tidak, pakai ID sebagai cadangan (fallback)
    use_token = 'token' in df.columns
    if not use_token:
        print("⚠️ PERINGATAN: Kolom 'token' tidak ditemukan! QR Code akan menggunakan 'id' biasa.")

    total_rows = len(df)
    print(f"\n--- Memulai Proses {total_rows} Tiket ---")

    for index, row in df.iterrows():
        # --- AMBIL DATA ---
        prefix = row['tribune']      # misal: 'A'
        ticket_id = row['id']        # misal: 'A-0001'
        gate_num = row['gate']       # misal: 7

        # Ambil Token (jika ada), jika tidak ada pakai ID
        token_value = row['token'] if use_token else row['id']

        if pd.notna(prefix) and pd.notna(gate_num):

            # Text Info: "Tribun 'A' - Gerbang 7"
            info_text = f"Tribun '{prefix}' - Gerbang {int(gate_num)}"

            # --- 1. Buat QR CODE (Isinya TOKEN) ---
            # Ini bagian pentingnya: QR Code berisi token rahasia, bukan ID urut
            qr = qrcode.QRCode(
                version=1,
                error_correction=qrcode.constants.ERROR_CORRECT_H, # High error correction
                box_size=10,
                border=2,
            )
            qr.add_data(token_value)
            qr.make(fit=True)

            qr_img = qr.make_image(fill_color="black", back_color="white")
            qr_img = qr_img.resize((500, 500))

            # --- 2. Buka Template ---
            try:
                ticket = Image.open(TEMPLATE_PATH).convert('RGB')
                img_width, img_height = ticket.size
            except FileNotFoundError:
                print(f"❌ ERROR: File template '{TEMPLATE_PATH}' tidak ditemukan!")
                break

            # --- 3. Tempel QR Code ---
            posisi_qr_x = (img_width - 500) // 2
            posisi_qr_y = 330
            ticket.paste(qr_img, (posisi_qr_x, posisi_qr_y))

            # --- 4. Tulis Teks ID & Info ---
            draw = ImageDraw.Draw(ticket)

            # A. Tulis ID Tiket (BESAR) -> Untuk Manusia
            posisi_ticket_id_y = 290
            draw.text((img_width / 2, posisi_ticket_id_y), ticket_id, font=font_ticket_id, fill='black', anchor='mm')

            # B. Tulis Info Gate (SEDANG)
            posisi_info_y = 830
            draw.text((img_width / 2, posisi_info_y), info_text, font=font_combined_info, fill='black', anchor='mm')

            # C. (Opsional) Tulis Token Kecil di pojok bawah untuk verifikasi manual darurat
            # draw.text((img_width / 2, 900), f"Ref: {token_value}", font=font_small, fill='gray', anchor='mm')

            # --- 5. Simpan File ---
            # Simpan berdasarkan folder tribun
            category_temp_folder = os.path.join(LOCAL_TEMP_DIR, prefix)
            os.makedirs(category_temp_folder, exist_ok=True)

            # Nama file tetap pakai ID agar mudah dicari panitia (misal: A-0001.png)
            # Token tidak perlu jadi nama file karena susah dibaca
            img_filename = f"{ticket_id}.png"
            img_path = os.path.join(category_temp_folder, img_filename)
            ticket.save(img_path)

            success_count += 1

        else:
            failed_count += 1
            failed_list.append(f"Baris {index}: Data tidak lengkap")

    print(f"✅ Selesai generate {success_count} tiket.")

    # --- 6. Pindahkan ke Google Drive / Output ---
    print("\n--- Memindahkan file ke output akhir... ---")
    shutil.move(LOCAL_TEMP_DIR, OUTPUT_DIR_NAME)
    print(f"📂 Lokasi akhir: {os.path.abspath(OUTPUT_DIR_NAME)}")

    # --- 7. Laporan ---
    print(f"\nTotal: {total_rows} | Berhasil: {success_count} | Gagal: {failed_count}")

except KeyError as e:
    print(f"❌ ERROR: Kolom {e} hilang. Pastikan pakai CSV hasil generate token sebelumnya.")
except Exception as e:
    print(f"❌ Error tak terduga: {e}")
    # Bersihkan folder temp jika error
    if os.path.exists(LOCAL_TEMP_DIR):
        shutil.rmtree(LOCAL_TEMP_DIR)

File CSV dibaca dari: /content/drive/MyDrive/G Colab/e-Ticket Pramuka Garuda/data-eksekusi.csv
Template dibaca dari: /content/drive/MyDrive/G Colab/e-Ticket Pramuka Garuda/template-ticket.png
Hasil disimpan di: /content/drive/MyDrive/G Colab/e-Ticket Pramuka Garuda/generated 3
✅ Font berhasil dimuat.

--- Memulai Proses 2176 Tiket ---
✅ Selesai generate 2176 tiket.

--- Memindahkan file ke output akhir... ---
📂 Lokasi akhir: /content/drive/MyDrive/G Colab/e-Ticket Pramuka Garuda/generated 3

Total: 2176 | Berhasil: 2176 | Gagal: 0


## Bulking Email

### Install Library PyDrive

In [ ]:
# Sel 6: Instal library Google Drive (PyDrive2)
!pip install PyDrive2 -q
print("✅ Library PyDrive2 berhasil diinstal.")

✅ Library PyDrive2 berhasil diinstal.


### Autentikasi

In [ ]:
# Sel 7: Autentikasi Google Drive untuk PyDrive
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Autentikasi pengguna Colab
# Ini akan memunculkan pop-up untuk login dan memberikan izin.
auth.authenticate_user()

# 2. Inisialisasi GoogleAuth dengan kredensial Colab
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

# 3. Buat objek drive PyDrive
drive = GoogleDrive(gauth)

print("✅ Autentikasi Google Drive (Metode PyDrive) Berhasil.")
print("   Sekarang Anda seharusnya memiliki izin untuk mengubah file.")

✅ Autentikasi Google Drive (Metode PyDrive) Berhasil.
   Sekarang Anda seharusnya memiliki izin untuk mengubah file.


### Ubah Permision Public

#### Atur Folder Yang akan Dipublikasi

In [ ]:
# --- KONFIGURASI ---
NAMA_FOLDER_GENERATED = "generated 3" # Sesuaikan jika beda
FILE_RINGKASAN = "data-eksekusi.csv"
PRINT_INTERVAL = 50 # <-- Lapor kemajuan setiap 50 file
# --------------------

#### Eksekusi Ubah File

In [ ]:
import time
import pandas as pd
import os

try:
    # Sekarang variabel FILE_RINGKASAN sudah dikenali
    df_ringkasan = pd.read_csv(FILE_RINGKASAN)
    print(f"✅ Berhasil membaca {FILE_RINGKASAN}...")
except FileNotFoundError:
    print(f"❌ ERROR: File '{FILE_RINGKASAN}' tidak ditemukan.")
    # Stop eksekusi jika file csv tidak ada
    raise

# --- 2. Cari ID Folder Utama ---
parent_folder_id = 'root'
# Mendapatkan path saat ini di Colab
current_path = os.getcwd().replace('/content/drive/MyDrive/', '')
folders = current_path.split('/')

# Mencari ID folder tempat script ini berada
for folder_name in folders:
    if not folder_name: continue # Skip jika string kosong
    file_list = drive.ListFile({'q': f"'{parent_folder_id}' in parents and title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
    if file_list:
        parent_folder_id = file_list[0]['id']
    else:
        # Fallback jika path tidak terdeteksi sempurna (misal di root)
        pass

print(f"Berada di folder ID: {parent_folder_id}")

# Mencari folder 'generated 5' di dalam folder tersebut
folder_list = drive.ListFile({'q': f"'{parent_folder_id}' in parents and title='{NAMA_FOLDER_GENERATED}' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()

if not folder_list:
    print(f"❌ ERROR: Folder '{NAMA_FOLDER_GENERATED}' tidak ditemukan.")
    raise FileNotFoundError(f"Folder '{NAMA_FOLDER_GENERATED}' tidak ditemukan")

id_folder_generated = folder_list[0]['id']
print(f"✅ Folder '{NAMA_FOLDER_GENERATED}' ditemukan (ID: {id_folder_generated})")
print("\nMemberi waktu 5 detik untuk sinkronisasi...")
time.sleep(5)

# --- 3. Looping (HANYA UBAH IZIN) ---
total_sukses = 0
total_rows = len(df_ringkasan)
print(f"\n--- Memulai proses perizinan {total_rows} file (ini akan memakan waktu)... ---")

for index, row in df_ringkasan.iterrows():
    # Pastikan nama kolom sesuai dengan CSV Anda ('id' dan 'tribune')
    kode_tiket = row['id']
    tribun = row['tribune']

    try:
        # 1. Cari Sub-folder (misal: A, B, C1)
        sub_folder_list = drive.ListFile({'q': f"'{id_folder_generated}' in parents and title='{tribun}' and trashed=false"}).GetList()
        if not sub_folder_list:
            continue # Skip jika folder tribun tidak ada

        id_sub_folder = sub_folder_list[0]['id']

        # 2. Cari File Gambar (misal: A-0001.png)
        file_list = drive.ListFile({'q': f"'{id_sub_folder}' in parents and title='{kode_tiket}.png' and trashed=false"}).GetList()
        if not file_list:
            continue # Skip jika file gambar belum ter-upload

        # 3. Ubah Izin (Share to Anyone with Link)
        file_id = file_list[0]['id']
        file_to_share = drive.CreateFile({'id': file_id})
        permission = { 'type': 'anyone', 'role': 'reader', 'withLink': True }
        file_to_share.InsertPermission(permission)

        total_sukses += 1

        # 4. Laporan Kemajuan
        if (index + 1) % PRINT_INTERVAL == 0 or (index + 1) == total_rows:
            print(f"    ... {total_sukses} / {total_rows} file berhasil diubah izinnya ...")

    except Exception as e:
        print(f"❌ ERROR saat memproses {kode_tiket}: {e}")

print(f"\n🎉 TUGAS SELESAI. Total {total_sukses} / {total_rows} file telah diubah izinnya menjadi Public.")

✅ Berhasil membaca data-eksekusi.csv...
Berada di folder ID: 1L74H19wdee2IBttnwsf01bg4dgk-7TcK
✅ Folder 'generated 3' ditemukan (ID: 1zvipqDO-XyqvQXeh8feGrO5GGy68CQ54)

Memberi waktu 5 detik untuk sinkronisasi...

--- Memulai proses perizinan 2176 file (ini akan memakan waktu)... ---
    ... 50 / 2176 file berhasil diubah izinnya ...
    ... 100 / 2176 file berhasil diubah izinnya ...
    ... 150 / 2176 file berhasil diubah izinnya ...
    ... 200 / 2176 file berhasil diubah izinnya ...
    ... 250 / 2176 file berhasil diubah izinnya ...
    ... 300 / 2176 file berhasil diubah izinnya ...
    ... 350 / 2176 file berhasil diubah izinnya ...
    ... 400 / 2176 file berhasil diubah izinnya ...
    ... 450 / 2176 file berhasil diubah izinnya ...
    ... 500 / 2176 file berhasil diubah izinnya ...
    ... 550 / 2176 file berhasil diubah izinnya ...
    ... 600 / 2176 file berhasil diubah izinnya ...
    ... 650 / 2176 file berhasil diubah izinnya ...
    ... 700 / 2176 file berhasil diubah 

### Csv Baru

#### Csv beserta Link

##### Atur Folder mana yang dieksekusi

In [ ]:
try:
    _ = drive # Cek apakah 'drive' ada
except NameError:
    print("❌ ERROR: Variabel 'drive' tidak ditemukan.")
    print("   Harap jalankan ulang Sel 7 (Autentikasi) terlebih dahulu!")
    raise

print("--- TUGAS 2: Mengambil Link dan Membuat CSV Final ---")

# --- KONFIGURASI ---
NAMA_FOLDER_GENERATED = "generated 3" # <-- Pastikan ini benar
FILE_RINGKASAN = "data-eksekusi.csv"
FILE_OUTPUT = "email-final.csv"
PRINT_INTERVAL = 50
# --------------------

--- TUGAS 2: Mengambil Link dan Membuat CSV Final ---


##### Eksekusi File Csv Baru dengan Link

In [ ]:
# (Asumsi Sel 7 dan 8 untuk Autentikasi dan Set Izin Publik sudah jalan)

link_list = []
first_file_id_to_check = None

try:
    # FILE_RINGKASAN adalah 'data-eksekusi.csv' dari sel sebelumnya
    df_ringkasan = pd.read_csv(FILE_RINGKASAN)
    print(f"Membaca {FILE_RINGKASAN}...")
except FileNotFoundError:
    print(f"❌ ERROR: File '{FILE_RINGKASAN}' tidak ditemukan.")
    raise
except KeyError as e:
    print(f"❌ ERROR: Kolom {e} tidak ada. Pastikan FILE_RINGKASAN adalah file final.")
    raise

# --- 1. Cari ID Folder Utama (Sama) ---
parent_folder_id = 'root'
current_path = os.getcwd().replace('/content/drive/MyDrive/', '')
folders = current_path.split('/')
for folder_name in folders:
    file_list = drive.ListFile({'q': f"'{parent_folder_id}' in parents and title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
    if file_list: parent_folder_id = file_list[0]['id']
    else: parent_folder_id = 'root'; break
id_folder_generated_list = drive.ListFile({'q': f"'{parent_folder_id}' in parents and title='{NAMA_FOLDER_GENERATED}' and trashed=false"}).GetList()
if not id_folder_generated_list:
    print(f"❌ ERROR: Folder '{NAMA_FOLDER_GENERATED}' tidak ditemukan.")
    raise FileNotFoundError(f"Folder '{NAMA_FOLDER_GENERATED}' tidak ditemukan")
id_folder_generated = id_folder_generated_list[0]['id']
print(f"✅ Folder '{NAMA_FOLDER_GENERATED}' ditemukan.")

# --- 2. Looping (HANYA AMBIL LINK) ---
total_sukses = 0
total_rows = len(df_ringkasan)
print(f"\n--- Memulai proses pengambilan {total_rows} link... ---")

for index, row in df_ringkasan.iterrows():

    # Membaca kolom 'id' dan 'tribune'
    id_tiket = row['id']
    tribune = row['tribune']

    try:
        # Mencari sub-folder
        sub_folder_list = drive.ListFile({'q': f"'{id_folder_generated}' in parents and title='{tribune}' and trashed=false"}).GetList()
        if not sub_folder_list:
            link_list.append(None)
            continue
        id_sub_folder = sub_folder_list[0]['id']

        # Mencari file
        file_list = drive.ListFile({'q': f"'{id_sub_folder}' in parents and title='{id_tiket}.png' and trashed=false"}).GetList()
        if not file_list:
            link_list.append(None)
            continue

        file_id = file_list[0]['id']
        if first_file_id_to_check is None:
            first_file_id_to_check = file_id

        # Logika Pengambilan Link
        file_to_get_link = drive.CreateFile({'id': file_id})
        file_to_get_link.FetchMetadata(fields='id,title,webViewLink,alternateLink,permissions')
        link = file_to_get_link.get('webViewLink')
        if link is None:
            link = file_to_get_link.get('alternateLink')

        link_list.append(link)

        if link:
            total_sukses += 1

        # Laporan Kemajuan (Progress Bar)
        if (index + 1) % PRINT_INTERVAL == 0 or (index + 1) == total_rows:
            print(f"     ... {total_sukses} / {index + 1} link diambil ...")

    except Exception as e:
        print(f"❌ ERROR saat mengambil link {id_tiket}: {e}")
        link_list.append(None)

print(f"✅ Proses pengambilan link selesai. Total {total_sukses} / {total_rows} link berhasil diambil.")

# ==========================================================
# === 3. Buat CSV Final (MODIFIKASI DI SINI) ===
# ==========================================================
# Tambahkan kolom 'link_tiket'
df_ringkasan['link_tiket'] = link_list

# BUANG kolom 'isCheckedIn' sebelum menyimpan
# 'errors='ignore'' -> agar skrip tidak error jika kolomnya sudah tidak ada
df_final_output = df_ringkasan.drop(columns=['isCheckedIn'], errors='ignore')

# Simpan DataFrame yang sudah bersih
df_final_output.to_csv(FILE_OUTPUT, index=False)
print(f"\n🎉 TUGAS 2 SELESAI! File '{FILE_OUTPUT}' telah dibuat (tanpa 'isCheckedIn').")
# ==========================================================

# --- Tampilkan 5 Data Pertama ---
print("\n--- 5 Baris Pertama dari File Hasil ---")
# Menampilkan dari df_final_output (yang sudah bersih)
print(df_final_output.head().to_markdown(index=False, numalign="left", stralign="left"))


# --- Verifikasi Izin Publik (Sama) ---
print("\n--- Memverifikasi Izin Publik ---")
if first_file_id_to_check:
    try:
        test_file = drive.CreateFile({'id': first_file_id_to_check})
        permissions = test_file.GetPermissions()
        is_public = False
        for perm in permissions:
            if perm.get('type') == 'anyone': is_public = True; break
        if is_public: print("✅ VERIFIKASI SUKSES: File pertama sudah 'Publik'.")
        else: print("❌ VERIFIKASI GAGAL: File masih 'Private'. (Pastikan Sel 8 sudah jalan)")
    except Exception as e:
        try:
            test_file.FetchMetadata(fields='permissions')
            permissions = test_file.get('permissions', [])
            is_public = False
            for perm in permissions:
                if perm.get('type') == 'anyone': is_public = True; break
            if is_public: print("✅ VERIFIKASI SUKSES: File pertama sudah 'Publik'.")
            else: print("❌ VERIFIKASI GAGAL: File masih 'Private'.")
        except Exception as e2:
            print(f"Gagal memverifikasi izin: {e2}")
else:
    print("⚠️ Tidak ada file yang ditemukan untuk diverifikasi.")

Membaca data-eksekusi.csv...
✅ Folder 'generated 3' ditemukan.

--- Memulai proses pengambilan 2176 link... ---
     ... 50 / 50 link diambil ...
     ... 100 / 100 link diambil ...
     ... 150 / 150 link diambil ...
     ... 200 / 200 link diambil ...
     ... 250 / 250 link diambil ...
     ... 300 / 300 link diambil ...
     ... 350 / 350 link diambil ...
     ... 400 / 400 link diambil ...
     ... 450 / 450 link diambil ...
     ... 500 / 500 link diambil ...
     ... 550 / 550 link diambil ...
     ... 600 / 600 link diambil ...
     ... 650 / 650 link diambil ...
     ... 700 / 700 link diambil ...
     ... 750 / 750 link diambil ...
     ... 800 / 800 link diambil ...
     ... 850 / 850 link diambil ...
     ... 900 / 900 link diambil ...
     ... 950 / 950 link diambil ...
     ... 1000 / 1000 link diambil ...
     ... 1050 / 1050 link diambil ...
     ... 1100 / 1100 link diambil ...
     ... 1150 / 1150 link diambil ...
     ... 1200 / 1200 link diambil ...
     ... 1250 / 

## Mock Data - Jalankan jika ingin membuat 3000 data dummy

In [ ]:
# Sel 1: Instal Faker (untuk data palsu)
!pip install faker -q

import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta

# --- 1. Konfigurasi ---
N_ROWS = 3000
ORIGINAL_FILE = 'tes-registrasi.csv' # Pastikan file ini ada di folder Anda
OUTPUT_FILE = 'mock_data_3000.csv'
CATEGORIES = ['CAT 1', 'CAT 2', 'CAT 3', 'CAT 4', 'CAT 5']

# Inisialisasi Faker (id_ID untuk nama/kota Indonesia)
fake = Faker('id_ID')

# --- 2. Baca Kolom dari File Asli ---
try:
    df_original = pd.read_csv(ORIGINAL_FILE)
    original_columns = df_original.columns
    print(f"Berhasil membaca struktur kolom dari: {ORIGINAL_FILE}")
except FileNotFoundError:
    print(f"❌ ERROR: File asli '{ORIGINAL_FILE}' tidak ditemukan.")
    print("Harap pastikan file CSV Anda sudah ada di direktori yang sama.")
    # Jika file tidak ada, hentikan eksekusi
    raise
except Exception as e:
    print(f"Error saat membaca file asli: {e}")
    raise

# --- 3. Generate Mock Data ---
print(f"Membuat {N_ROWS} baris data tiruan...")
data_list = []

# Tentukan rentang waktu (misal, 30 hari terakhir)
base_time = datetime.now() - timedelta(days=30)

for i in range(N_ROWS):
    # Buat timestamp yang urut tapi acak
    # (Setiap pendaftar baru muncul 0-10 menit setelah sebelumnya)
    row_time = base_time + timedelta(minutes=i*random.randint(1, 10))

    row_data = {
        'Timestamp': row_time.strftime('%Y/%m/%d %I:%M:%S %p GMT+7'),
        'Username': fake.email(),
        'Dengan ini saya menyatakan menyetujui Syarat dan Ketentuan yang berlaku, dan siap untuk kooperatif dengan Petugas serta menerima konsekuensi yang timbul di kemudian hari.': 'Setuju',
        'Nama Lengkap': fake.name(),
        'Jenis Kelamin': random.choice(['Pria', 'Wanita']),
        'Orangtua/Wali dari ...': fake.name(),
        'Domisili Kecamatan': fake.city(),
        'Pangkalan Gudep Peserta Pelantikan': random.choice(['Perguruan Tinggi', 'SMA N 1', 'SMK N 2', 'Umum', 'Lainnya']),
        'Kontak WhatsApp': f"628{random.randint(100000000, 999999999)}", # Format 628...
        'Tribun yang Kakak Pilih': random.choice(CATEGORIES)
    }
    data_list.append(row_data)

# --- 4. Buat DataFrame dan Simpan ---
mock_df = pd.DataFrame(data_list)

# Pastikan urutan kolom SAMA PERSIS dengan file asli
mock_df = mock_df[original_columns]

# Simpan ke file CSV baru
mock_df.to_csv(OUTPUT_FILE, index=False)

print(f"\n✅ BERHASIL! File '{OUTPUT_FILE}' telah dibuat.")
print(f"Total baris data: {len(mock_df)}")
print("\n--- Pengecekan Distribusi Tribun ---")
# Menampilkan hitungan setiap kategori untuk bukti
print(mock_df['Tribun yang Kakak Pilih'].value_counts())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.0 MB/s eta 0:00:00
Berhasil membaca struktur kolom dari: tes-registrasi.csv
Membuat 3000 baris data tiruan...

✅ BERHASIL! File 'mock_data_3000.csv' telah dibuat.
Total baris data: 3000

--- Pengecekan Distribusi Tribun ---
Tribun yang Kakak Pilih
CAT 4    653
CAT 5    618
CAT 1    591
CAT 2    583
CAT 3    555
Name: count, dtype: int64
